In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from qml import qcnn as q
import layers as cl # custom layers 
import copy

ModuleNotFoundError: No module named 'layers'

In [ ]:
## read / import data 
num_qubits = 8
training_fname = "./qcnns_for_phase_recog/data/dataset_n=8_train.txt"
test_fname = "./qcnns_for_phase_recog/data/dataset_n=8_test.txt"

def read_eigenvectors(file):
    with open(file, 'r+') as f:
        textData = f.readlines()

        h_vals = []
        for i in range(len(textData)):
            h1h2, eigenvector = textData[i].split("_")

            h_vals.append(tuple(map(float, h1h2[1: -1].split(', '))))
            textData[i] = eigenvector

        return h_vals, np.loadtxt(textData, dtype=complex)

h1h2_train, train_data = read_eigenvectors(training_fname)
h1h2_test, test_data = read_eigenvectors(test_fname)

labels = np.zeros(40)

for index, h1h2 in enumerate(h1h2_train):
    h1, h2 = h1h2
    if h1 <= 1: 
        labels[index] = 1.0


In [ ]:
## Model 
my_qcnn = q.Qcnn(num_qubits)

## Add Custom layer: 
legacy_fully_connected_layer = cl.get_legacy_fc_layer(num_qubits // 3)  # because we will be pooling out 2/3 of the qubits 
my_qcnn.Layers[legacy_fully_connected_layer.name] = legacy_fully_connected_layer # we have to add it to our model class variable


## def structure: 
my_qcnn.add_layer(my_qcnn.Layers["legacy_conv4_layer"], kwargs={"label":"C4_1"} )
my_qcnn.add_layer(my_qcnn.Layers["legacy_conv_layer"], kwargs={"label": "C2"})
# my_qcnn.add_layer(my_qcnn.Layers["legacy_conv_layer"], kwargs={"label": "C3", "start_index": 1})
# my_qcnn.add_layer(my_qcnn.Layers["legacy_conv_layer"], kwargs={"label": "C4", "start_index": 2})
my_qcnn.add_layer(my_qcnn.Layers["legacy_pool_layer"], kwargs={"label": "P1", 
                                                               "update_active_qubits": {"group_len": 3, "target": 1}})
                                                              # after this we have pooled out 2 out of every 3 qubits 

my_qcnn.add_layer(my_qcnn.Layers[legacy_fully_connected_layer.name], kwargs={})  # this is the custom layer we added earlier ! 

## Initialize parameters: 
my_qcnn.initialize_params(random=True)
initial_params = copy.deepcopy(my_qcnn.params)

## visual check : 
# circ = my_qcnn.generate_circ(my_qcnn.params)
# circ.draw(reverse_bits=True)

In [ ]:
## Learning as described in paper: 
learning_rate = 100000  # intial value was 10 but this quantity doesn't learn fast enough ! 
successive_loss = 1.0 # initialize to arbitrary value > 10^-5
loss_lst = []  # initialize 
iteration_num = 1 

while (abs(successive_loss) > 1e-5) and (iteration_num < 10):
    pred = my_qcnn.forward(train_data, my_qcnn.params.copy())
    loss = my_qcnn.mse_loss(pred, labels)
    
    print("---- Iteration : {}, Loss {} ----------------------".format(iteration_num, loss))
    
    if iteration_num == 1:
        pass 
    
    else: 
        successive_loss = loss - loss_lst[-1]       
        if successive_loss < 0: 
            learning_rate *= 1.05  # if loss decreases, increase learning rate by 5%
        else:
            learning_rate /= 2  # if it gets bigger, decrease learning rate by 50% 
            
    grad_mat = my_qcnn.compute_grad(train_data, labels)
#     grad_mat = my_qcnn.compute_grad_w_mp(train_data, labels)  # with multi processing 
    my_qcnn.update_params(grad_mat, learning_rate)
    
    loss_lst.append(loss)
    iteration_num += 1 
    
# params = copy.deepcopy(my_qcnn.params)

In [ ]:
# print(params)  ## extracting parameters to save future training time 

In [ ]:
## Using model on test data (graph visualization) :
predictions = my_qcnn.forward(test_data, my_qcnn.params.copy())
print('got predictions!')

In [ ]:
flipped_pred_mat = predictions.reshape((64, 32), order='F')
pred_mat = []
for row_index in np.arange(len(flipped_pred_mat) - 1, -1, -1):
    pred_mat.append(flipped_pred_mat[row_index])

pred_mat = np.array(pred_mat)

heat_map = plt.imshow(pred_mat, cmap = 'autumn', interpolation = 'nearest')
plt.colorbar(heat_map, label="Exp_val X")
plt.title( "2-D Heat Map" )
plt.xlabel('h1/J')
plt.ylabel('h2/J')
plt.show()

In [ ]:
# print(initial_params)  

In [ ]:
## heat_map for initial params (graph visualization) :
initial_predictions = my_qcnn.forward(test_data, initial_params)
print('got predictions!')

In [ ]:
flipped_pred_mat = initial_predictions.reshape((64, 64), order='F')
pred_mat = []
for row_index in np.arange(len(flipped_pred_mat) - 1, -1, -1):
    pred_mat.append(flipped_pred_mat[row_index])

pred_mat = np.array(pred_mat)


# plot heat map 
heat_map = plt.imshow(pred_mat, cmap = 'autumn', interpolation = 'nearest')
plt.colorbar(heat_map, label="Exp_val X")
plt.title( "2-D Heat Map" )
plt.xlabel('h1/J')
plt.ylabel('h2/J')

# plot phase boundaries
h1_vals = [0.1000, 0.2556, 0.4111, 0.5667, 0.7222, 0.8778, 1.0333, 1.1889, 1.3444, 1.5000]
anti_ferro_mag_boundary = [-1.004, -1.0009, -1.024, -1.049, -1.079, -1.109, -1.154,  -1.225, -1.285, -1.35]
para_mag_boundary = [0.8439, 0.6636, 0.5033, 0.3631, 0.2229, 0.09766, -0.02755, -0.1377, -0.2479, -0.3531]

plt.plot(h1_vals, anti_ferro_mag_boundary, label="Antiferromagnetics")
plt.plot(h1_vals, para_mag_boundary, label="Paramagnetic")

plt.show()

In [ ]:
## Loss plot: 
x_axis = range(len(loss_lst))

plt.plot(x_axis, loss_lst)
plt.title('Training Loss over Epoches')
plt.xlabel('Epoches')
plt.ylabel("Loss")
plt.show()